# Natural Frequency of a Mass-Spring-Damper System
This notebook interactively demonstrates the behavior of a single-degree-of-freedom mass-spring-damper system. The system is governed by the second-order differential equation:
$$ m\ddot{x} + c\dot{x} + kx = 0 $$
We can explore how changing the physical parameters affects the system's response.

### Key Parameters
* **Natural Frequency ($\omega_n$):** The frequency at which the system oscillates without any driving or damping forces. It's determined by mass and stiffness: $\omega_n = \sqrt{k/m}$.
* **Damping Ratio ($\zeta$):** A dimensionless number describing how oscillations decay after a disturbance.
    * $\zeta < 1$: **Underdamped** (oscillates before settling)
    * $\zeta = 1$: **Critically Damped** (fastest return to zero without oscillation)
    * $\zeta > 1$: **Overdamped** (slow, non-oscillating return to zero)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, Layout

# --- Core Physics and Plotting Functions ---

def solve_and_plot(m, k, c):
    """
    Solves the system's response and plots both time and frequency domains.
    """
    # 1. Calculate Key Parameters
    # Ensure m and k are not zero to avoid division errors
    if m <= 0 or k <= 0:
        print("Mass (m) and Stiffness (k) must be positive.")
        return
        
    w_n = np.sqrt(k / m)  # Natural Frequency (rad/s)
    zeta = c / (2 * np.sqrt(m * k)) if (m > 0 and k > 0) else 0 # Damping Ratio

    # --- Setup for Plotting ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5.5))
    fig.suptitle(f"Mass-Spring-Damper System Response (m={m:.1f} kg, k={k:.0f} N/m, c={c:.1f} Ns/m)", fontsize=16)

    # --- Plot 1: Time-Domain Response (How it oscillates after release) ---
    def model(y, t, m, k, c):
        x, v = y
        dxdt = v
        dvdt = (-c * v - k * x) / m
        return [dxdt, dvdt]

    # Initial conditions: displace by 1 unit and release from rest
    y0 = [1.0, 0.0]
    t_end = 15.0  # Extend time to see decay of heavily damped systems
    t = np.linspace(0, t_end, 500)
    
    sol = odeint(model, y0, t, args=(m, k, c))
    x_t = sol[:, 0]

    ax1.plot(t, x_t, 'b-', linewidth=2.5)
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.set_xlabel('Time (s)', fontsize=12)
    ax1.set_ylabel('Displacement (m)', fontsize=12)
    ax1.set_title(f'Time Response (ζ = {zeta:.2f})', fontsize=14)
    ax1.axhline(0, color='black', lw=1)
    ax1.set_xlim(0, t_end)
    ax1.set_ylim(-1.1, 1.1)

    # Add text to classify the damping type
    if zeta < 1:
        ax1.text(0.95, 0.95, 'Underdamped', transform=ax1.transAxes, ha='right', va='top', fontsize=14, color='green', fontweight='bold')
    elif abs(zeta - 1.0) < 0.01:
        ax1.text(0.95, 0.95, 'Critically Damped', transform=ax1.transAxes, ha='right', va='top', fontsize=14, color='blue', fontweight='bold')
    else:
        ax1.text(0.95, 0.95, 'Overdamped', transform=ax1.transAxes, ha='right', va='top', fontsize=14, color='red', fontweight='bold')


    # --- Plot 2: Frequency-Domain Response (Resonance Curve) ---
    # Define a range of driving frequencies to test
    driving_freq = np.linspace(0.01, 2.5 * w_n, 1000)
    
    # Standard formula for steady-state amplitude
    Amplitude = 1 / np.sqrt((k - m * driving_freq**2)**2 + (c * driving_freq)**2)

    ax2.plot(driving_freq, Amplitude, 'r-', linewidth=2.5)
    ax2.axvline(w_n, color='gray', linestyle='--', lw=2, label=f'Natural Freq. ωn = {w_n:.2f} rad/s')
    ax2.grid(True, linestyle='--', alpha=0.6)
    ax2.set_xlabel('Driving Frequency (rad/s)', fontsize=12)
    ax2.set_ylabel('Steady-State Amplitude (m)', fontsize=12)
    ax2.set_title('Frequency Response (Resonance)', fontsize=14)
    ax2.legend(fontsize=12)
    ax2.set_xlim(0, 2.5 * w_n)
    
    # Dynamically set a sensible y-axis limit to prevent extreme spikes
    # from making the graph unreadable for low damping.
    peak_amplitude = np.max(Amplitude)
    ax2.set_ylim(0, min(peak_amplitude * 1.2, 0.5)) # Capped at 0.5 for visual stability

    plt.tight_layout(rect=[0, 0.03, 1, 0.94])
    plt.show()

# --- Create the Interactive Sliders ---
# Use a wider layout for the sliders
slider_layout = Layout(width='60%')
interact(
    solve_and_plot,
    m=FloatSlider(min=1.0, max=20.0, step=0.5, value=10.0, description='Mass (m)', layout=slider_layout),
    k=FloatSlider(min=10.0, max=2000.0, step=10, value=1000.0, description='Stiffness (k)', layout=slider_layout),
    c=FloatSlider(min=0.0, max=250.0, step=1.0, value=20.0, description='Damping (c)', layout=slider_layout)
);

interactive(children=(FloatSlider(value=10.0, description='Mass (m)', layout=Layout(width='60%'), max=20.0, mi…

### Things to Try
1.  **Find Critical Damping:** For the default mass (m=10) and stiffness (k=1000), what value of the damping coefficient `c` makes the system critically damped ($\zeta = 1$)?
2.  **Observe Resonance:** Set the damping `c` to a very low value (e.g., 5.0). Notice how the peak of the *Frequency Response* curve becomes extremely high at the natural frequency. This is resonance.
3.  **Effect of Mass:** How does doubling the mass `m` affect the natural frequency $\omega_n$?